# Optimizing Model Parameters

모델을 학습시키는 것은 반복적 과정이다. 각 반복에서 모델은 예측을 하고, 이 예측에 대한 loss를 계산하고, derivative를 수집한 뒤 gradient descent를 이용해 parameter를 최적화한다.

## Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

Hyperparameters는 adjustable parameters로 최적화 과정에서 제어할 수 있는 것들이다. 하이퍼 파라미터 튜닝에 대해 더 알아보려면 [여기](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)를 살펴보자.

튜닝을 위해 다음과 같은 하이퍼파리미터를 정의한다.
- 에폭 수
- 배치 사이즈
- 학습률

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

하이퍼파라미터를 설정한 후에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다. 이 때 각 반복을 에폭이라고 한다.

한 에폭은 다음과 같이 구성된다.
- The Train Loop - 전 학습 데이터 셋을 iterate하고 최적 파라미터로 수렴하려고 한다.
- The Validation/Test Loop - 전 테스트 데이터 셋을 iterate하고 모델 성능이 향상되고 있는지 확인한다.

## Loss Function

손실 함수에는 다음과 같은 것들이 있다.
- nn.MSELoss(Mean Square Error)
- nn.NLLLoss(Negative Log Likelihood)
- nn.CrossEntropyLoss(nn.LogSoftmax와 nn.NLLLoss를 합친 것)

In [3]:
# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

SGD, ADAM, RMSProp 등 다양한 optimizer들을 [여기](https://pytorch.org/docs/stable/optim.html)서 확인할 수 있다.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

training loop에서 최적화는 다음과 같은 과정으로 이루어진다.
- optimizer.zero_grad()로 모델 파라미터의 gradients를 초기화.
- loss.backward()로 backpropagate
- gradients를 통해 parameters를 조절하도록 optimizer.step()

## Full Implementation

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    # Set the model training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practice
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.931778  [   64/60000]
loss: 1.905345  [ 6464/60000]
loss: 1.798592  [12864/60000]
loss: 1.834560  [19264/60000]
loss: 1.708631  [25664/60000]
loss: 1.680793  [32064/60000]
loss: 1.679808  [38464/60000]
loss: 1.595789  [44864/60000]
loss: 1.606664  [51264/60000]
loss: 1.503358  [57664/60000]
Test Error: 
 Accuracy: 59.3%, Avg loss: 1.523480 

Epoch 2
-------------------------------
loss: 1.586879  [   64/60000]
loss: 1.557206  [ 6464/60000]
loss: 1.411592  [12864/60000]
loss: 1.482002  [19264/60000]
loss: 1.346609  [25664/60000]
loss: 1.359735  [32064/60000]
loss: 1.357467  [38464/60000]
loss: 1.294370  [44864/60000]
loss: 1.321269  [51264/60000]
loss: 1.226008  [57664/60000]
Test Error: 
 Accuracy: 62.3%, Avg loss: 1.252660 

Epoch 3
-------------------------------
loss: 1.330443  [   64/60000]
loss: 1.314949  [ 6464/60000]
loss: 1.151062  [12864/60000]
loss: 1.258128  [19264/60000]
loss: 1.121972  [25664/60000]
loss: 1.160143  [32064/600